#### SuperTrend calculation optimisation
The supertrend I downloaded seems to work but is painfully slow. Looking at the code it uses a lot of loops, which are very inefficient. I will use it as a correct but slow method, and develop a faster one.
Check out this implementation:
https://github.com/arkochhar/Technical-Indicators/blob/master/indicator/indicators.py

I think we can combine the two loops into one, although I may be wrong. Lets implement it and test it out.

In [1]:
import pandas as pd
import numpy as np
from signals import *
from indicators.indicator import SuperTrend

In [5]:
df = load_ticker("EURUSD", 60)
display(df)

,Open,High,Low,Close,TR,ATR_10,ST_10_3,STX_10_3
2019-01-21 23:00:00,1.1369,1.1370,1.1367,1.1368,0.0003,0.000000,0.000000,0.0
2019-01-22 00:00:00,1.1367,1.1370,1.1367,1.1367,0.0003,0.000000,0.000000,0.0
2019-01-22 01:00:00,1.1367,1.1371,1.1365,1.1368,0.0006,0.000000,0.000000,0.0
2019-01-22 02:00:00,1.1368,1.1369,1.1365,1.1366,0.0004,0.000000,0.000000,0.0
2019-01-22 03:00:00,1.1367,1.1369,1.1361,1.1361,0.0008,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
2020-01-29 19:00:00,1.1000,1.1019,1.0997,1.1016,0.0022,0.000845,1.098526,1.0
2020-01-29 20:00:00,1.1016,1.1019,1.1003,1.1004,0.0016,0.000920,1.098526,1.0
2020-01-29 21:00:00,1.1004,1.1009,1.1001,1.1009,0.0008,0.000908,1.098526,1.0
2020-01-29 22:00:00,1.1009,1.1011,1.1007,1.1009,0.0004,0.000857,1.098526,1.0


In [7]:
SuperTrend(df, 10, 3)
display(df)

,Open,High,Low,Close,TR,ATR_10,ST_10_3,STX_10_3
2019-01-21 23:00:00,1.1369,1.1370,1.1367,1.1368,0.0003,0.000000,0.000000,0.0
2019-01-22 00:00:00,1.1367,1.1370,1.1367,1.1367,0.0003,0.000000,0.000000,0.0
2019-01-22 01:00:00,1.1367,1.1371,1.1365,1.1368,0.0006,0.000000,0.000000,0.0
2019-01-22 02:00:00,1.1368,1.1369,1.1365,1.1366,0.0004,0.000000,0.000000,0.0
2019-01-22 03:00:00,1.1367,1.1369,1.1361,1.1361,0.0008,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...
2020-01-29 19:00:00,1.1000,1.1019,1.0997,1.1016,0.0022,0.000845,1.098526,1.0
2020-01-29 20:00:00,1.1016,1.1019,1.1003,1.1004,0.0016,0.000920,1.098526,1.0
2020-01-29 21:00:00,1.1004,1.1009,1.1001,1.1009,0.0008,0.000908,1.098526,1.0
2020-01-29 22:00:00,1.1009,1.1011,1.1007,1.1009,0.0004,0.000857,1.098526,1.0


In [ ]:
def EMA(df, base, target, period, alpha=False):
    """
    Function to compute Exponential Moving Average (EMA)
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        base : String indicating the column name from which the EMA needs to be computed from
        target : String indicates the column name to which the computed data needs to be stored
        period : Integer indicates the period of computation in terms of number of candles
        alpha : Boolean if True indicates to use the formula for computing EMA using alpha (default is False)
        
    Returns :
        df : Pandas DataFrame with new column added with name 'target'
    """

    con = pd.concat(
        [df[:period][base].rolling(window=period).mean(), df[period:][base]]
    )

    if alpha == True:
        # (1 - alpha) * previous_val + alpha * current_val where alpha = 1 / period
        df[target] = con.ewm(alpha=1 / period, adjust=False).mean()
    else:
        # ((current_val - previous_val) * coeff) + previous_val where coeff = 2 / (period + 1)
        df[target] = con.ewm(span=period, adjust=False).mean()

    df[target].fillna(0, inplace=True)
    return df


def ATR(df, period, ohlc=["Open", "High", "Low", "Close"]):
    """
    Function to compute Average True Range (ATR)
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        period : Integer indicates the period of computation in terms of number of candles
        ohlc: List defining OHLC Column names (default ['Open', 'High', 'Low', 'Close'])
        
    Returns :
        df : Pandas DataFrame with new columns added for 
            True Range (TR)
            ATR (ATR_$period)
    """
    atr = "ATR_" + str(period)

    if atr in df.columns:
        return df

    # Compute true range only if it is not computed and stored earlier in the df
    if not "TR" in df.columns:
        df["h-l"] = df[ohlc[1]] - df[ohlc[2]]
        df["h-yc"] = abs(df[ohlc[1]] - df[ohlc[3]].shift())
        df["l-yc"] = abs(df[ohlc[2]] - df[ohlc[3]].shift())

        df["TR"] = df[["h-l", "h-yc", "l-yc"]].max(axis=1)

        df.drop(["h-l", "h-yc", "l-yc"], inplace=True, axis=1)

    # Compute EMA of true range using ATR formula after ignoring first row
    EMA(df, "TR", atr, period, alpha=True)

    return df


def SuperTrend(df, period, multiplier, ohlc=["Open", "High", "Low", "Close"]):
    """
    Function to compute SuperTrend
    
    Args :
        df : Pandas DataFrame which contains ['date', 'open', 'high', 'low', 'close', 'volume'] columns
        period : Integer indicates the period of computation in terms of number of candles
        multiplier : Integer indicates value to multiply the ATR
        ohlc: List defining OHLC Column names (default ['Open', 'High', 'Low', 'Close'])
        
    Returns :
        df : Pandas DataFrame with new columns added for 
            True Range (TR), ATR (ATR_$period)
            SuperTrend (ST_$period_$multiplier)
            SuperTrend Direction (STX_$period_$multiplier)
    """
    
    atr = "ATR_" + str(period)
    st = "ST_" + str(period) + "_" + str(multiplier)
    stx = "STX_" + str(period) + "_" + str(multiplier)
    
    if stx in df.columns:
        return df
    
    ATR(df, period, ohlc=ohlc)


    """
    SuperTrend Algorithm :
    
        BASIC UPPERBAND = (HIGH + LOW) / 2 + Multiplier * ATR
        BASIC LOWERBAND = (HIGH + LOW) / 2 - Multiplier * ATR
        
        FINAL UPPERBAND = IF( (Current BASICUPPERBAND < Previous FINAL UPPERBAND) or (Previous Close > Previous FINAL UPPERBAND))
                            THEN (Current BASIC UPPERBAND) ELSE Previous FINALUPPERBAND)
        FINAL LOWERBAND = IF( (Current BASIC LOWERBAND > Previous FINAL LOWERBAND) or (Previous Close < Previous FINAL LOWERBAND)) 
                            THEN (Current BASIC LOWERBAND) ELSE Previous FINAL LOWERBAND)
        
        SUPERTREND = IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close <= Current FINAL UPPERBAND)) THEN
                        Current FINAL UPPERBAND
                    ELSE
                        IF((Previous SUPERTREND = Previous FINAL UPPERBAND) and (Current Close > Current FINAL UPPERBAND)) THEN
                            Current FINAL LOWERBAND
                        ELSE
                            IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close >= Current FINAL LOWERBAND)) THEN
                                Current FINAL LOWERBAND
                            ELSE
                                IF((Previous SUPERTREND = Previous FINAL LOWERBAND) and (Current Close < Current FINAL LOWERBAND)) THEN
                                    Current FINAL UPPERBAND
    """

    # Compute basic upper and lower bands
    df["basic_ub"] = (df[ohlc[1]] + df[ohlc[2]]) / 2 + multiplier * df[atr]
    df["basic_lb"] = (df[ohlc[1]] + df[ohlc[2]]) / 2 - multiplier * df[atr]

    # Compute final upper and lower bands
    df["final_ub"] = 0.00
    df["final_lb"] = 0.00
    # Set the Supertrend value
    df[st] = 0.00
    for i in range(period, len(df)):
        df["final_ub"].iat[i] = (
            df["basic_ub"].iat[i]
            if df["basic_ub"].iat[i] < df["final_ub"].iat[i - 1]
            or df[ohlc[3]].iat[i - 1] > df["final_ub"].iat[i - 1]
            else df["final_ub"].iat[i - 1]
        )
        df["final_lb"].iat[i] = (
            df["basic_lb"].iat[i]
            if df["basic_lb"].iat[i] > df["final_lb"].iat[i - 1]
            or df[ohlc[3]].iat[i - 1] < df["final_lb"].iat[i - 1]
            else df["final_lb"].iat[i - 1]
        )
        df[st].iat[i] = (
            df["final_ub"].iat[i]
            if df[st].iat[i - 1] == df["final_ub"].iat[i - 1]
            and df[ohlc[3]].iat[i] <= df["final_ub"].iat[i]
            else df["final_lb"].iat[i]
            if df[st].iat[i - 1] == df["final_ub"].iat[i - 1]
            and df[ohlc[3]].iat[i] > df["final_ub"].iat[i]
            else df["final_lb"].iat[i]
            if df[st].iat[i - 1] == df["final_lb"].iat[i - 1]
            and df[ohlc[3]].iat[i] >= df["final_lb"].iat[i]
            else df["final_ub"].iat[i]
            if df[st].iat[i - 1] == df["final_lb"].iat[i - 1]
            and df[ohlc[3]].iat[i] < df["final_lb"].iat[i]
            else 0.0
        )

    # Mark the trend direction up/down
    df[stx] = np.where((df[st] > 0.00), np.where((df[ohlc[3]] < df[st]), -1, 1), np.NaN)

    # Remove basic and final bands from the columns
    df.drop(["basic_ub", "basic_lb", "final_ub", "final_lb"], inplace=True, axis=1)

    df.fillna(0, inplace=True)

    return df

In [ ]:
df = load_ticker("EURUSD", 60)
display(df)

In [ ]:
%%timeit
SuperTrend(df, 10, 4)

In [ ]:
display(df)